# Regression to Predict the runs scored given the type of bowlers expected to bowl and the state of the match

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('../../Data/selected_data/processed_data.csv')

# merging medium bowlers to fast
data.loc[data['bowler_type'] == 'Left arm Medium', 'bowler_type'] = 'Left arm Fast'
data.loc[data['bowler_type'] == 'Right arm Medium', 'bowler_type'] = 'Right arm Fast'
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217242 entries, 0 to 217241
Data columns (total 32 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   batter               217242 non-null  object 
 1   bowler               217242 non-null  object 
 2   non_striker          217242 non-null  object 
 3   runs_by_bat          217242 non-null  int64  
 4   extra_runs           217242 non-null  int64  
 5   total_runs_delivery  217242 non-null  int64  
 6   current_team_total   217242 non-null  int64  
 7   runs_remain          217242 non-null  float64
 8   batter_runs          217242 non-null  int64  
 9   balls_faced          217242 non-null  int64  
 10  wickets_fallen       217242 non-null  int64  
 11  extra_type           217242 non-null  object 
 12  delivery             217242 non-null  float64
 13  over                 217242 non-null  int64  
 14  wicket_type          217242 non-null  object 
 15  player_out       

In [31]:
def getPlayerScores(player_name: str, innings: list[int] = [1, 2] ) -> pd.DataFrame:
    # Get the data for BKG Mendis if batter is BKG Mendis or non-striker is BKG Mendis
	player_data = data.loc[
		((data['batter'] == player_name) | (data['non_striker'] == player_name)) & (data['innings'].isin(innings))
	]

	player_data.head()

	# group data by match_id
	gp = player_data.groupby('match_id')
	cols = [ 'batter', 'non_striker', 'batter_runs', 'balls_faced', 'wicket_type', 'won', 'innings', 'over', 'delivery', 'wickets_fallen', 'bowling_team', 'venue']
	player_scores= gp.last().loc[:, cols]

	# get the first ball he faced or at non-striker
	first_ball = gp.first().loc[:, ['over', 'delivery', 'wickets_fallen']]
	first_ball['first_ball'] = (first_ball['over'] * 6 + first_ball['delivery']).astype(int)

	player_scores['first_ball'] = first_ball['first_ball']
	player_scores['wickets_fallen'] = first_ball['wickets_fallen']

	# when BKG Mendis is the non-striker when the last ball was bowled
	# The batter_runs and balls_faced are not his, but the on_strike batter's
	# So, we need to get the last ball he faced
	# he might not even have faced a ball

	# get the last ball he faced

	matches_non_striker = player_scores[player_scores['non_striker'] == player_name].index
	
	# Sometimes the player might not even have faced a single ball
	# Eg: Afghanistan_Sri Lanka_2022-11-01 MD Shanaka not out on the non strikers end

	player_scores.loc[matches_non_striker, ['batter_runs', 'balls_faced']] = [0, 0]
	
	
	# get the last batter == player_name row from gp data
	gp = player_data[(player_data['batter'] == player_name) & (player_data['match_id'].isin(matches_non_striker))].groupby(['match_id'])
	last_batter_scores = gp.last()[['batter_runs', 'balls_faced']]	
	
	# update the rows with non_striker with correct values
	player_scores.update(last_batter_scores)
	
	



	# adding new features
	# strike rate
	player_scores['strike_rate'] = round(player_scores['batter_runs'] / player_scores['balls_faced'] * 100, 2)
	player_scores['out'] = player_scores['wicket_type'] != '0'
	player_scores['last_ball'] = (player_scores['over'] * 6 + player_scores['delivery']).astype(int)

	# drop over and delivery
	player_scores.drop(['over', 'delivery'], inplace=True, axis=1)


	# concatenating the remaining bowler types number to the dataset
	matches = data[data['match_id'].isin(player_scores.index)]
	# matches = matches[matches['batting_team'] == 'Sri Lanka']
	cols = ['match_id', 'batter', 'non_striker', 'bowler_type', 'batter_runs', 'balls_faced', 'wicket_type', 'won', 'innings', 'over', 'delivery', 'wickets_fallen']
	matches = matches[cols]
	matches['ball_number'] = (matches['over'] * 6 + matches['delivery']).astype(int)
	matches.drop(['over', 'delivery'], inplace=True, axis = 1)


	def filter_by_player_and_ball_number(group):
		player_data = group[group['batter'] == player_name]
		
		if player_data.empty:
			return player_data.drop('match_id', axis=1)
		
		first_ball_number = player_data['ball_number'].iloc[0]

		# return player_data[player_data['ball_number'] > first_ball_number].drop('match_id', axis=1) # This is for number of balls faced
		
		# fixed error should be greater or equal
		remaining = group[group['ball_number'] >= first_ball_number].drop('match_id', axis=1) # return the remianing number of deliveries for each type
		return remaining

	gp = matches.groupby('match_id').apply(filter_by_player_and_ball_number)
	remaining_ball_types = gp.groupby('match_id')['bowler_type'].value_counts().unstack(fill_value = 0)
	remaining_ball_types = remaining_ball_types.reset_index()

	player_scores = player_scores.merge(remaining_ball_types, how='left', on='match_id')

	### Attaching the player_v_bowler results to this dataset
	player_vs_bowler = get_player_v_bowlers(player_name)
	


	bowler_types = ['Left arm Fast', 'Right arm Fast', 'Left arm Orthodox', 'Left arm Wrist spin', 'Right arm Legbreak', 'Right arm Offbreak']
	for bowler_type in bowler_types:
			if bowler_type in player_vs_bowler.index:
				player_scores[f'{bowler_type} Expected Runs'] = player_vs_bowler.loc[bowler_type, 'strike_rate'] / 100 *  player_scores[bowler_type]
				player_scores[f'{bowler_type} Expected Wickets'] = player_scores[bowler_type] / player_vs_bowler.loc[bowler_type, 'deliveries_per_wicket']
                player_scores[f'{bowler_type} Strike Rate'] = player_vs_bowler.loc[bowler_type, 'strike_rate']
                player_scores[f'{bowler_type} Deliveries Per Wicket'] =  player_vs_bowler.loc[bowler_type, 'deliveries_per_wicket']
            else:
				player_scores[bowler_type] = 0
				player_scores[f'{bowler_type} Expected Runs'] = 0
				player_scores[f'{bowler_type} Expected Wickets'] = 0
                player_scores[f'{bowler_type} Strike Rate'] = 0
                player_scores[f'{bowler_type} Deliveries Per Wicket'] = 0

    
	# player_scores.fillna(0, inplace=True)
	return player_scores

def get_player_v_bowlers(player_name: str, innings = [1, 2]) -> pd.DataFrame:
	player_data = data.loc[
			(data['batter'] == player_name) & (data['innings'].isin(innings))
		]

	player_data.head()

	# Convert all medium bowlers to fast
	player_data.loc[player_data['bowler_type'] == 'Left arm Medium', 'bowler_type'] = 'Left arm Fast'
	player_data.loc[player_data['bowler_type'] == 'Right arm Medium', 'bowler_type'] = 'Right arm Fast'


	player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out') 


	cols = ['batter', 'non_striker', 'runs_by_bat', 'out', 'won', 'innings', 'over', 'delivery','bowler_type']
	player_data = player_data[cols]

	gp = player_data.groupby('bowler_type')

	player_v_bowler = pd.DataFrame()
	player_v_bowler['strike_rate'] = round(gp['runs_by_bat'].mean() * 100, 3)
	player_v_bowler['strike_rate_std'] = gp['runs_by_bat'].std()
	player_v_bowler['wickets'] = gp['out'].sum()
	player_v_bowler['deliveries_per_wicket'] = round(1 / gp['out'].mean(), 3)
	player_v_bowler['deliveries'] = gp.size()
	return player_v_bowler



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 96)

In [7]:
merged_df = pd.DataFrame()

for player in data['batter'].unique():
    print("Analyzing Player", player)
    
    player_scores = getPlayerScores(player)
    player_scores['batter'] = player
    player_scores.drop('non_striker', inplace=True, axis = 1)
    merged_df = pd.concat([merged_df, player_scores])



Analyzing Player AJ Finch
Analyzing Player M Klinger


/tmp/ipykernel_1276995/1574441845.py:80: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gp = matches.groupby('match_id').apply(filter_by_player_and_ball_number)
/tmp/ipykernel_1276995/1574441845.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
/tmp/ipykernel_1276995/1574441845.py:80: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping col

Analyzing Player TM Head
Analyzing Player MC Henriques
Analyzing Player AJ Turner


/tmp/ipykernel_1276995/1574441845.py:80: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gp = matches.groupby('match_id').apply(filter_by_player_and_ball_number)
/tmp/ipykernel_1276995/1574441845.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
/tmp/ipykernel_1276995/1574441845.py:80: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping col

Analyzing Player JP Faulkner
Analyzing Player N Dickwella
Analyzing Player WU Tharanga


...

/tmp/ipykernel_1276995/1574441845.py:80: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gp = matches.groupby('match_id').apply(filter_by_player_and_ball_number)
/tmp/ipykernel_1276995/1574441845.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
/tmp/ipykernel_1276995/1574441845.py:80: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping col

Analyzing Player Rafatullah Mohmand
Analyzing Player Imran Khan (1)
Analyzing Player RLB Rambukwella
Analyzing Player KMC Bandara


/tmp/ipykernel_1276995/1574441845.py:80: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gp = matches.groupby('match_id').apply(filter_by_player_and_ball_number)
/tmp/ipykernel_1276995/1574441845.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
/tmp/ipykernel_1276995/1574441845.py:80: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping col

Analyzing Player TB de Bruyn
Analyzing Player ST de Silva
Analyzing Player Shuvagata Hom
Analyzing Player Muktar Ali
Analyzing Player Khurram Manzoor


/tmp/ipykernel_1276995/1574441845.py:80: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  gp = matches.groupby('match_id').apply(filter_by_player_and_ball_number)
/tmp/ipykernel_1276995/1574441845.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_data['out'] = (player_data['wicket_type'] != '0') & (player_data['wicket_type'] != 'run out')
/tmp/ipykernel_1276995/1574441845.py:80: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping col

In [8]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14957 entries, 0 to 1
Data columns (total 32 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   match_id                              14957 non-null  object 
 1   batter                                14957 non-null  object 
 2   batter_runs                           14957 non-null  int64  
 3   balls_faced                           14957 non-null  int64  
 4   wicket_type                           14957 non-null  object 
 5   won                                   14957 non-null  float64
 6   innings                               14957 non-null  int64  
 7   wickets_fallen                        14957 non-null  int64  
 8   bowling_team                          14957 non-null  object 
 9   venue                                 14957 non-null  object 
 10  first_ball                            14957 non-null  int64  
 11  strike_rate             